In [453]:
import os
import sklearn
import pandas as pd
import csv

In [454]:
n_gram_value = 5

In [455]:
def get_ngrams(input_list, n):    
    return zip(*[input_list[i:] for i in range(n)])


In [456]:
def getAllPossibleWords(file_path): 
    possible_words=[]
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                possible_words.append(word)     
   # possible_words = map(lambda x : unidecode.unidecode(x),possible_words)
    for c in ['`',"[","]","(",")","\"",".","''",","]:
        possible_words = map(lambda x: x.replace(c,""),possible_words)
        
    possible_words = map(lambda x: x.replace('&','And'),possible_words)
    #possible_words = map(lambda x: x.replace('of','Of'),possible_words)
    possible_words=filter(lambda x:len(x)>0,possible_words)    
    
    return possible_words

In [457]:
def getListFromFiles(filename):
    file_path = os.path.join(os.getcwd(),filename)
    info_list = []
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                info_list.append(word)
    return info_list

In [458]:
def writeToCsv(processed_list):
    with open("output-sreejita.csv",'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        #for row in processed_list:
        wr.writerows(processed_list)

In [459]:
def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1): 
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple)                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            elif(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple) 
                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):  
                    
                    #prune unigrams 
                    if(i==1):                        
                        if(ngram[0] not in suffix_list):
                            if(word_number> word_count+1 and ngram[0] not in common_words):
                                #add the unigram to the list if the next word begins with lower case
                                if(possible_words[word_count+1][0].islower()):
                                    preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])
                    
                    #prune common words
                    elif(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])                   

                    
                     
            word_count = word_count+1
    return preprocessed_list    

In [460]:
def generateFeatures(filename, processed_list,possible_words):
    
    company_suffixes = getListFromFiles("company-suffixes.txt")
    company_prefixes = getListFromFiles("company-prefixes.txt")    
    company_list=[]
    final_processed_list = []
    splittedFilename = filename.split("-")
    i=0
    #print "possible_words- "  
    #print possible_words
    for company in processed_list:
        i+=1
        #Feature 1 : name has any prefix or suffix of a company 
        #if company name has any one of the suffixes or prefixes set it to true 
        hasCompanyid=0        
        if(company[0].split()[-1] in company_suffixes or company[0].split()[0] in company_prefixes):
            hasCompanyid = 1
            company_list.append(company[0])            
        #company.append(hasCompanyid)
        
        
        #has Company name at the beginning ?
        #Feature 2: name occurs at the beginning of a company name
        hasCompanyNameFirst = 0
        l1= [x for x in company[0].split() if x not in company_prefixes]
        l2=[]
        for comp in company_list:    
            if(comp.split()[0] in company_prefixes):
                l2.append(" ".join(comp.split()[1:]))
            else:
                l2.append(comp)
        if(any(word in l2 for word in l1)):
            hasCompanyNameFirst = 1
        elif(any(company[0].split()[0]== word.split()[0] for word in company_list)):
            hasCompanyNameFirst = 1 
        
            
        
        #has Company name as a substring ?
        #Feature 3: name is a substring in the company name
        hasNameSubstring=0
        for comp in company_list:
            l1 = company[0].split()
            l2 = comp.split()
            if(len(set(l1)&set(l2)) > 0 ):
                hasNameSubstring=1       
       
         
        
        #Feature 4: name in filename
        isInFilename = 0
        splittedInstance = company[0].split()
        for sp in splittedInstance:
            if sp.lower() in splittedFilename:
                isInFilename = 1 
        
        
        #Feature 5-y - Check neighbouring words - check 3 words before and 3 words after
        starting_loc = company[2]
        ending_loc = company[3]
        words_before_starting_loc = []
        words_after_ending_loc = []
        j= starting_loc-1
        count = 1
        while j >= 0 and count <= 3:
            words_before_starting_loc.append(possible_words[company[2]-count])
            j-=1
            count+=1
        
        j = ending_loc+1
        count = 1
        while j < len(possible_words) and count <= 3:
            words_after_ending_loc.append(possible_words[company[3]+count])
            j+=1
            count+=1
        
        #Feature 5 - Check for word "based" before the company
        has_based = 0
        for word in words_before_starting_loc:
            if "based" in word.lower():
                has_based = 1
        
        #Feature 6 "'s" is after the company
        has_apostrophe_s = 0
        for word in words_after_ending_loc:
            if "'s" in word.lower():
                has_apostrophe_s = 1
        '''
        #Feature 7 "," is after/before the company
        has_comma = 0
        for word in words_after_ending_loc:
            if "," in word.lower():
                has_comma = 1
        
        for word in words_before_starting_loc:
            if "," in word.lower():
                has_comma = 1
        '''
        #Feature "shares before or after the company
        has_shares = 0
        for word in words_after_ending_loc:
            if "shares" in word.lower():
                has_shares = 1
              
        for word in words_before_starting_loc:
            if "shares" in word.lower():
                has_shares = 1 
                
        #Feature 8 designations before/after company
        desig = open('designations.txt').read().split('\n')
        desig_lower = [x.lower() for x in desig]
        has_desig = 0
        for word in words_after_ending_loc:
            if word.lower() in desig_lower:
                has_desig = 1
              
        for word in words_before_starting_loc:
            if word.lower() in desig_lower:
                has_desig = 1 
         
        #Feature 9 "said/told" in the before company
        has_said = 0
        for word in words_before_starting_loc:
            if "said" in word.lower() or "told" in word.lower():
                has_said = 1
        
        '''
        #Feature 10 Prune away Examples if previous/next word starts with a Capital
        surrounded_capital = 0
        if len(words_before_starting_loc) > 0:
            prev_word = words_before_starting_loc[0]
            if len(prev_word) > 0 and prev_word[0].isupper():
                surrounded_capital = 1
        if len( words_after_ending_loc) > 0:
            next_word = words_after_ending_loc[0]
            if len(next_word) > 0 and next_word[0].isupper():
                surrounded_capital = 1
      
          
                
        
        print "company:- " + company[0]
        print "words_after_ending_loc"
        print words_after_ending_loc
        print "words_before_starting_loc"
        print words_before_starting_loc '''
        
        company.extend([hasCompanyid, hasCompanyNameFirst, hasNameSubstring, isInFilename, has_based, has_apostrophe_s, has_shares, has_desig, has_said])
        
            

In [461]:
base_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(base_directory,"Dataset","Testing_1")
file_list = os.listdir(data_directory)
possible_words=[]
processed_list=[]
count=0
#doc_id = 101
for file in file_list:
    relevant_filename = file[4:-4]
    doc_id = file[0:3]
    #print(doc_id)
    #if(int(doc_id)!=95):
    #    continue    
    possible_words = getAllPossibleWords(os.path.join(data_directory,file))     
    
    #start from here .. processessed list has the structure[[word,doc_id,word_count_start,word_count_end,label]]
    ''''processed_list.append(preprocessing(possible_words,doc_id))
    writeToCsv(processed_list)'''
    processed_list_for_doc = preprocessing(possible_words,doc_id)
    #print processed_list_for_doc
    generateFeatures(relevant_filename, processed_list_for_doc,possible_words)
    processed_list.extend(processed_list_for_doc)
    #writeToCsv(processed_list) 
  

In [462]:
ngram = ('<markup>Gazprom', 'Lukoil</markup>')
if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
    print "hallo"


hallo


In [463]:
["markup" in word for word in ngram].count(True)==2

True

In [464]:
import pycountry
country_names = []
for country in pycountry.countries:
    country_names.append(country.name)
    if hasattr(country,'common_name'):
        country_names.append(country.common_name)
    if hasattr(country, 'official_name'):
        country_names.append(country.official_name)

country_names.append("America")
country_names.append("Latin America")
country_names.append("Korea")
country_names.append("South Korea")
country_names.append("North Korea")
country_names.append("U.S.")
country_names.append("Russia")
country_names.append("Siberia")

if "Russia" in country_names:
    print "YES"
    
continent_list = ['Asia', 'Africa', 'South America', 'North America', 'Antarctica', 'Europe', 'Australia']

YES


In [465]:
cities = []
with open('world-cities.csv', 'r') as inp:
        for row in csv.reader(inp):
            if(row and row[3] != "" and row[0]!="name"):
                cities.append(row[0])

cities.append('Bombay')
cities.append('New York')
cities.append('Montreal')
cities.append('Frankfurt')
cities.append('Zurich')
if "Bedford" in cities:
    print "YES"

us_states = open('us-states.txt').read().split('\n')
ethni_list = open('ethnicities.txt').read().split('\n')

YES


In [466]:
desig = open('designations.txt').read().split('\n')
desig_lower = [x.lower() for x in desig]

print desig_lower

['chief executive officer', 'chief financial officer', 'co-chief executive', 'chief investment officer', 'analyst', 'ceo', 'cio', 'cto', 'cfo', 'manager', 'director', 'president', 'vice-president', 'vice president', 'chief', 'executive', 'officer', 'staff', 'secretary', 'attorney', 'jr', 'sr', 'officials', 'officers', 'analysts', 'managers', 'directors', 'co-chief', 'co chief', 'commissioner', 'minister', 'magistrate', 'judge', 'chairman']


In [467]:
pruned_pos = open('pruned_pos.txt').read().split('\n')
stock_x = open('stock-exchange-list.txt').read().split('\n')
unigram_prune = open('unigram-prune-neg.txt').read().split('\n')
special_char = [';', ',', ':', '?']
prune = open('pruning-neg.txt').read().split('\n') 
prune_lower = [x.lower() for x in prune]
suffixes = open('company-suffixes.txt').read().split('\n')
prefixes = open('company-prefixes.txt').read().split('\n')
rows = 0
print len(processed_list)
for doc in processed_list:
    rows += len(doc)


3595


In [468]:
print pruned_pos
new_proc_list = []
pruned_list = []

for inst in processed_list:
    if inst[0] not in pruned_pos and not inst[0].endswith(tuple(suffixes)) and not inst[0].startswith(tuple(prefixes)): 
        if inst[0] in country_names or inst[0] in continent_list or inst[0] in cities or inst[0] in stock_x or inst[0] in us_states:
            pruned_list.append(inst)
        elif "'s" in inst[0]:
            pruned_list.append(inst)
        elif inst[0].endswith('n') and inst[0][:-1] in country_names or str in continent_list:
            pruned_list.append(inst)
        elif "based" in inst[0]:
            pruned_list.append(inst)
        elif any(d in inst[0].lower().split() for d in desig_lower):
            pruned_list.append(inst)
        elif any(p in inst[0].lower().split() for p in prune_lower):
            pruned_list.append(inst)
        elif '(' in inst[0] or ')' in inst[0]:
            pruned_list.append(inst)
        elif any(c in inst[0] for c in special_char):
            pruned_list.append(inst)
        elif inst[0].startswith("A ") or inst[0].startswith("An ") or inst[0].startswith("And ") or inst[0].startswith("This"):
             pruned_list.append(inst)
        elif inst[0].endswith("And") or inst[0].endswith("This"):
             pruned_list.append(inst)
        elif len(inst[0].split()) == 1:
            if len(inst[0]) == 1:
                pruned_list.append(inst)
            elif inst[0] in unigram_prune or inst[0] in ethni_list:
                pruned_list.append(inst)
            elif inst[0] == inst[0].upper():
                pruned_list.append(inst)
            elif inst[0] in suffixes or inst[0] in prefixes:
                pruned_list.append(inst)
            else:
                new_proc_list.append(inst)
        else:
            new_proc_list.append(inst)
    else:
        new_proc_list.append(inst)
            
                
print len(pruned_list)
for eg in pruned_list:
    if eg[4] == 1:
        print eg[0] + " " + str(eg[1])
#writeToCsv(processed_list)
print len(new_proc_list)
print len(pruned_list) + len(new_proc_list)
#print pruned_list



['ACE', "Sam's Club", 'ICBC', 'KKR', 'CSN', 'Tata', 'Wheeling', "River's Edge Pharmaceuticals LLC", "River's Edge", 'RWE', 'Fremont', 'News Corp', 'GM', 'Liberty', 'Financial Services Authority', 'FSA', 'NZOG', 'Norilsk', 'NTP', 'MPC', 'PPR', 'THQ', 'Nokia', 'RCS', 'Intercontinental Exchange Inc', 'EON', 'RHB', 'WCM', 'Macclesfield', 'Carrefour', 'KPI', 'TXU', 'ECI', 'BP', 'TNK-BP', 'Imperial', 'Zug', 'TDF', 'TMK', "Ivernia Inc's", 'SAIC', 'GE', 'PICC', 'Saku', 'Winterthur', 'ZKB', "Rupert Murdoch's News Corp", 'PIK', 'VTB', 'UBS', 'IGT', 'BHP', 'BBH', 'China Banking Regulatory Commission', 'Compton', 'Centennial', 'NPD', 'HSBC', "Victoria's Secret", 'MAN', 'MBIA', 'Freeport', 'MGIC', 'PMI', 'Stillwater', "Dr Reddy's Laboratories Ltd", "Royal Mail's", 'Cia Siderurgica Nacional SA', 'Cia Siderurgica Nacional', 'Vector Hospitality Plc', 'Vector', 'Victory', 'Bedford', 'Equity Office Properties']
1664
1931
3595


In [469]:
with open("test_pruned_sr.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in new_proc_list:
        writer.writerow(val)